# Historical Linguistics with AlteruPhono: Complete Workflow Demo

This notebook demonstrates advanced historical linguistics workflows using AlteruPhono, including:
- Comparative reconstruction
- Language evolution simulation
- Phonological distance analysis
- Sound change rule discovery
- Cross-linguistic typology

**Prerequisites**: Basic knowledge of historical linguistics and phonology

In [ ]:
# Import required libraries
import sys
import os
sys.path.insert(0, os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))

import alteruphono
from alteruphono.phonology.sound_change import (
    SoundChangeEngine, 
    SoundChangeRule, 
    FeatureChangeRule,
    RuleSet
)
from alteruphono.phonology.sound_change.rules import FeatureChange, ChangeType
from alteruphono.phonology.sound_change.environments import PhonologicalEnvironment
from alteruphono.phonology.sound_v2 import Sound
from alteruphono.phonology.feature_systems import get_feature_system, convert_between_systems

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import networkx as nx

# Set up plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("AlteruPhono Historical Linguistics Demo")
print("=====================================")
print(f"AlteruPhono version: {alteruphono.__version__ if hasattr(alteruphono, '__version__') else 'development'}")

## 1. Comparative Reconstruction Workflow

We'll reconstruct a proto-language from descendant forms using systematic correspondences.

In [ ]:
# Define cognate sets from related languages
cognate_sets = {
    'water': {
        'Language_A': ['w', 'a', 't', 'a'],
        'Language_B': ['b', 'a', 'd', 'a'],  
        'Language_C': ['v', 'a', 't', 'a']
    },
    'fire': {
        'Language_A': ['f', 'i', 'r', 'a'],
        'Language_B': ['p', 'i', 'r', 'a'],
        'Language_C': ['f', 'i', 'r', 'a']
    },
    'stone': {
        'Language_A': ['s', 't', 'o', 'n'],
        'Language_B': ['s', 't', 'o', 'n'],
        'Language_C': ['ʃ', 't', 'o', 'n']
    },
    'path': {
        'Language_A': ['p', 'a', 'θ'],
        'Language_B': ['p', 'a', 't'],
        'Language_C': ['p', 'a', 't']
    }
}

def extract_correspondences(cognate_sets):
    """Extract systematic sound correspondences from cognate sets."""
    correspondences = defaultdict(list)
    
    for meaning, languages in cognate_sets.items():
        # Get maximum length for alignment
        max_len = max(len(word) for word in languages.values())
        
        # Align cognates (simple position-based alignment)
        for pos in range(max_len):
            sounds_at_pos = []
            for lang, word in languages.items():
                if pos < len(word):
                    sounds_at_pos.append((lang, word[pos]))
            
            if len(sounds_at_pos) >= 2:  # Need at least 2 languages
                # Create correspondence tuple
                correspondence = tuple(sound for lang, sound in sorted(sounds_at_pos))
                correspondences[correspondence].append((meaning, pos))
    
    return correspondences

correspondences = extract_correspondences(cognate_sets)

print("Systematic Sound Correspondences:")
print("=" * 40)
for corr, contexts in correspondences.items():
    if len(contexts) > 1:  # Only show recurrent correspondences
        lang_a, lang_b, lang_c = corr
        context_str = ', '.join(f"{meaning}:{pos}" for meaning, pos in contexts)
        print(f"A:{lang_a} ~ B:{lang_b} ~ C:{lang_c} ({context_str})")

In [ ]:
# Reconstruct proto-forms using the comparative method
def reconstruct_proto_form(correspondence_set):
    """Reconstruct proto-sound from correspondence set."""
    # Simple reconstruction heuristics
    sounds = set(correspondence_set)
    
    # If all languages agree, reconstruct the common sound
    if len(sounds) == 1:
        return list(sounds)[0]
    
    # Apply reconstruction principles
    # Example: voiceless stops are often more conservative
    voiceless_stops = {'p', 't', 'k'}
    for sound in sounds:
        if sound in voiceless_stops:
            return sound
    
    # Default: take the most common sound
    from collections import Counter
    return Counter(correspondence_set).most_common(1)[0][0]

def reconstruct_proto_language(cognate_sets):
    """Reconstruct proto-forms for all cognate sets."""
    proto_language = {}
    
    for meaning, languages in cognate_sets.items():
        # Get all language forms
        forms = list(languages.values())
        max_len = max(len(form) for form in forms)
        
        proto_form = []
        for pos in range(max_len):
            sounds_at_pos = []
            for form in forms:
                if pos < len(form):
                    sounds_at_pos.append(form[pos])
            
            if sounds_at_pos:
                proto_sound = reconstruct_proto_form(sounds_at_pos)
                proto_form.append(proto_sound)
        
        proto_language[meaning] = proto_form
    
    return proto_language

proto_forms = reconstruct_proto_language(cognate_sets)

print("\nProto-Language Reconstruction:")
print("=" * 35)
for meaning, proto_form in proto_forms.items():
    proto_str = ' '.join(proto_form)
    print(f"*{proto_str} '{meaning}'")
    
    # Show reflexes in daughter languages
    for lang, form in cognate_sets[meaning].items():
        form_str = ' '.join(form)
        print(f"  {lang}: {form_str}")
    print()

## 2. Language Evolution Simulation

Simulate the evolution of the reconstructed proto-language into the attested daughter languages.

In [ ]:
# Define sound change rules for each language branch
engine = SoundChangeEngine(feature_system_name='unified_distinctive')

# Language A changes: relatively conservative
lang_a_rules = [
    # θ > θ (no change)
    SoundChangeRule(name="a_conservative", source_pattern="θ", target_pattern="θ")
]

# Language B changes: more innovative
lang_b_rules = [
    # f > p (initial strengthening)
    SoundChangeRule(name="b_f_to_p", source_pattern="f", target_pattern="p", environment="# _"),
    # w > b (fortition)
    SoundChangeRule(name="b_w_to_b", source_pattern="w", target_pattern="b"),
    # t > d / V_V (intervocalic voicing)
    SoundChangeRule(name="b_t_to_d", source_pattern="t", target_pattern="d", environment="V _ V"),
    # θ > t (stopping)
    SoundChangeRule(name="b_theta_to_t", source_pattern="θ", target_pattern="t")
]

# Language C changes: moderate innovation
lang_c_rules = [
    # w > v (partial fortition)
    SoundChangeRule(name="c_w_to_v", source_pattern="w", target_pattern="v"),
    # s > ʃ / _ t (assimilation)
    SoundChangeRule(name="c_s_to_sh", source_pattern="s", target_pattern="ʃ", environment="_ t"),
    # θ > t (stopping)
    SoundChangeRule(name="c_theta_to_t", source_pattern="θ", target_pattern="t")
]

# Create rule sets
lang_a_evolution = RuleSet(rules=lang_a_rules, iterative=False)
lang_b_evolution = RuleSet(rules=lang_b_rules, iterative=False)
lang_c_evolution = RuleSet(rules=lang_c_rules, iterative=False)

def simulate_evolution(proto_forms, rule_sets):
    """Simulate evolution of proto-language into daughter languages."""
    results = {}
    
    for lang_name, rules in rule_sets.items():
        results[lang_name] = {}
        
        for meaning, proto_form in proto_forms.items():
            # Convert to Sound objects
            sounds = [Sound(s, 'unified_distinctive') for s in proto_form]
            
            # Apply evolution rules
            result = engine.apply_rule_set(rules, sounds)
            
            # Extract final form
            evolved_form = [s.grapheme() for s in result.final_sequence]
            results[lang_name][meaning] = evolved_form
    
    return results

# Simulate evolution
rule_sets = {
    'Simulated_A': lang_a_evolution,
    'Simulated_B': lang_b_evolution,
    'Simulated_C': lang_c_evolution
}

simulated_results = simulate_evolution(proto_forms, rule_sets)

print("Language Evolution Simulation:")
print("=" * 35)
for meaning in proto_forms.keys():
    proto_str = ' '.join(proto_forms[meaning])
    print(f"*{proto_str} '{meaning}'")
    
    # Show simulated vs. actual forms
    for lang in ['A', 'B', 'C']:
        actual_form = ' '.join(cognate_sets[meaning][f'Language_{lang}'])
        simulated_form = ' '.join(simulated_results[f'Simulated_{lang}'][meaning])
        match = "✓" if actual_form == simulated_form else "✗"
        print(f"  {lang}: {actual_form} (simulated: {simulated_form}) {match}")
    print()

## 3. Phonological Distance Analysis

Calculate phonological distances between languages for classification purposes.

In [ ]:
def calculate_lexical_distance(lang1_forms, lang2_forms, feature_system='tresoldi_distinctive'):
    """Calculate average phonological distance between two languages."""
    total_distance = 0
    word_count = 0
    
    for meaning in lang1_forms.keys():
        if meaning in lang2_forms:
            word1 = lang1_forms[meaning]
            word2 = lang2_forms[meaning]
            
            # Calculate word-level distance
            word_distance = calculate_word_distance(word1, word2, feature_system)
            total_distance += word_distance
            word_count += 1
    
    return total_distance / word_count if word_count > 0 else 0

def calculate_word_distance(word1, word2, feature_system='tresoldi_distinctive'):
    """Calculate phonological distance between two words."""
    # Convert to Sound objects
    sounds1 = [Sound(s, feature_system) for s in word1]
    sounds2 = [Sound(s, feature_system) for s in word2]
    
    # Align words (simple approach: pad shorter word)
    max_len = max(len(sounds1), len(sounds2))
    
    # Calculate segment-by-segment distances
    total_distance = 0
    for i in range(max_len):
        if i < len(sounds1) and i < len(sounds2):
            # Both have sounds at this position
            distance = sounds1[i].distance_to(sounds2[i])
            total_distance += distance
        else:
            # One word is shorter - penalize insertion/deletion
            total_distance += 1.0  # Maximum distance penalty
    
    return total_distance / max_len  # Normalize by length

# Calculate distance matrix
languages = ['Language_A', 'Language_B', 'Language_C']
distance_matrix = np.zeros((len(languages), len(languages)))

for i, lang1 in enumerate(languages):
    for j, lang2 in enumerate(languages):
        if i != j:
            # Extract language-specific forms
            lang1_forms = {meaning: forms[lang1] for meaning, forms in cognate_sets.items()}
            lang2_forms = {meaning: forms[lang2] for meaning, forms in cognate_sets.items()}
            
            distance = calculate_lexical_distance(lang1_forms, lang2_forms)
            distance_matrix[i][j] = distance

# Create distance DataFrame
distance_df = pd.DataFrame(distance_matrix, index=languages, columns=languages)

print("Phonological Distance Matrix:")
print("=" * 35)
print(distance_df.round(3))

# Visualize distance matrix
plt.figure(figsize=(8, 6))
sns.heatmap(distance_df, annot=True, cmap='viridis_r', square=True, 
            fmt='.3f', cbar_kws={'label': 'Phonological Distance'})
plt.title('Phonological Distance Matrix')
plt.tight_layout()
plt.show()

In [ ]:
# Build phylogenetic tree from distance matrix
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.spatial.distance import squareform

# Convert to condensed distance matrix
condensed_distances = squareform(distance_matrix)

# Perform hierarchical clustering
linkage_matrix = linkage(condensed_distances, method='average')

# Plot dendrogram
plt.figure(figsize=(10, 6))
dendrogram(linkage_matrix, labels=languages, leaf_rotation=45)
plt.title('Phonological Distance-Based Language Tree')
plt.xlabel('Languages')
plt.ylabel('Distance')
plt.tight_layout()
plt.show()

# Analyze clustering results
print("\nPhylogenetic Analysis:")
print("=" * 25)
print(f"Closest languages: {languages[np.unravel_index(np.argmin(distance_matrix + np.eye(len(languages))*999), distance_matrix.shape)]}")
print(f"Most distant languages: {languages[np.unravel_index(np.argmax(distance_matrix), distance_matrix.shape)]}")

## 4. Sound Change Rule Discovery

Automatically discover sound change rules from comparative data.

In [ ]:
def discover_sound_changes(proto_forms, daughter_forms):
    """Automatically discover sound change rules from comparative data."""
    discovered_rules = []
    
    # Collect all sound changes
    changes = defaultdict(list)
    
    for meaning in proto_forms.keys():
        if meaning in daughter_forms:
            proto_word = proto_forms[meaning]
            daughter_word = daughter_forms[meaning]
            
            # Align words and find changes
            max_len = max(len(proto_word), len(daughter_word))
            
            for i in range(max_len):
                proto_sound = proto_word[i] if i < len(proto_word) else None
                daughter_sound = daughter_word[i] if i < len(daughter_word) else None
                
                if proto_sound and daughter_sound and proto_sound != daughter_sound:
                    # Extract context
                    left_context = proto_word[i-1] if i > 0 else '#'
                    right_context = proto_word[i+1] if i < len(proto_word)-1 else '#'
                    
                    change_key = (proto_sound, daughter_sound)
                    context = (left_context, right_context)
                    changes[change_key].append(context)
    
    # Analyze patterns
    for (source, target), contexts in changes.items():
        if len(contexts) >= 2:  # Need multiple instances
            # Find common contexts
            left_contexts = [ctx[0] for ctx in contexts]
            right_contexts = [ctx[1] for ctx in contexts]
            
            # Simple rule discovery: find most common context
            from collections import Counter
            common_left = Counter(left_contexts).most_common(1)[0][0]
            common_right = Counter(right_contexts).most_common(1)[0][0]
            
            # Create rule description
            if common_left == '#' and common_right == '#':
                environment = "everywhere"
            elif common_left == '#':
                environment = f"word-initially (before {common_right})"
            elif common_right == '#':
                environment = f"word-finally (after {common_left})"
            else:
                environment = f"after {common_left}, before {common_right}"
            
            rule = {
                'source': source,
                'target': target,
                'environment': environment,
                'frequency': len(contexts),
                'examples': contexts[:3]  # Show first 3 examples
            }
            
            discovered_rules.append(rule)
    
    return discovered_rules

# Discover rules for Language B
lang_b_forms = {meaning: forms['Language_B'] for meaning, forms in cognate_sets.items()}
lang_b_rules = discover_sound_changes(proto_forms, lang_b_forms)

print("Discovered Sound Changes (Proto → Language B):")
print("=" * 50)
for rule in sorted(lang_b_rules, key=lambda x: x['frequency'], reverse=True):
    print(f"{rule['source']} → {rule['target']} / {rule['environment']}")
    print(f"  Frequency: {rule['frequency']} instances")
    print(f"  Examples: {rule['examples'][:2]}")
    print()

# Discover rules for Language C
lang_c_forms = {meaning: forms['Language_C'] for meaning, forms in cognate_sets.items()}
lang_c_rules = discover_sound_changes(proto_forms, lang_c_forms)

print("Discovered Sound Changes (Proto → Language C):")
print("=" * 50)
for rule in sorted(lang_c_rules, key=lambda x: x['frequency'], reverse=True):
    print(f"{rule['source']} → {rule['target']} / {rule['environment']}")
    print(f"  Frequency: {rule['frequency']} instances")
    print(f"  Examples: {rule['examples'][:2]}")
    print()

## 5. Cross-Linguistic Phonological Analysis

Analyze phonological inventories and typological patterns using the Tresoldi comprehensive system.

In [ ]:
# Define phonological inventories for different language types
language_inventories = {
    'Minimal_System': {
        'consonants': ['p', 't', 'k', 'm', 'n', 's'],
        'vowels': ['a', 'i', 'u']
    },
    'Average_System': {
        'consonants': ['p', 'b', 't', 'd', 'k', 'g', 'm', 'n', 'ŋ', 'f', 's', 'ʃ', 'l', 'r'],
        'vowels': ['a', 'e', 'i', 'o', 'u']
    },
    'Complex_System': {
        'consonants': ['p', 'b', 't', 'd', 'k', 'g', 'q', 'ʔ', 'm', 'n', 'ŋ', 'f', 'v', 'θ', 'ð', 's', 'z', 'ʃ', 'ʒ', 'x', 'ɣ', 'h', 'l', 'r', 'j', 'w'],
        'vowels': ['a', 'æ', 'e', 'ɛ', 'i', 'ɪ', 'o', 'ɔ', 'u', 'ʊ', 'ə']
    },
    'Click_System': {
        'consonants': ['p', 't', 'k', 'm', 'n', 's', 'l', 'ǀ', 'ǁ', 'ǃ'],  # Includes clicks
        'vowels': ['a', 'e', 'i', 'o', 'u']
    }
}

def analyze_inventory_typology(inventory, system_name='tresoldi_distinctive'):
    """Analyze typological properties of a phonological inventory."""
    tresoldi_system = get_feature_system(system_name)
    
    analysis = {
        'total_sounds': 0,
        'consonants': 0,
        'vowels': 0,
        'places_of_articulation': set(),
        'manners_of_articulation': set(),
        'rare_features': defaultdict(int),
        'feature_complexity': 0
    }
    
    all_sounds = inventory['consonants'] + inventory['vowels']
    analysis['total_sounds'] = len(all_sounds)
    
    rare_features = ['click', 'ejective', 'implosive', 'creaky', 'breathy']
    
    for sound_char in all_sounds:
        if tresoldi_system.has_grapheme(sound_char):
            sound = Sound(sound_char, system_name)
            
            # Count major classes
            if sound.get_feature_value('consonantal') > 0:
                analysis['consonants'] += 1
            if sound.get_feature_value('syllabic') > 0:
                analysis['vowels'] += 1
            
            # Collect places of articulation
            place_features = ['labial', 'coronal', 'dorsal', 'glottal']
            for place in place_features:
                if sound.has_feature(place) and sound.get_feature_value(place) > 0:
                    analysis['places_of_articulation'].add(place)
            
            # Collect manners of articulation  
            manner_features = ['nasal', 'continuant', 'lateral', 'strident']
            for manner in manner_features:
                if sound.has_feature(manner) and sound.get_feature_value(manner) > 0:
                    analysis['manners_of_articulation'].add(manner)
            
            # Count rare features
            for rare_feat in rare_features:
                if sound.has_feature(rare_feat) and sound.get_feature_value(rare_feat) > 0:
                    analysis['rare_features'][rare_feat] += 1
            
            # Calculate feature complexity (number of active features)
            active_features = sum(1 for feat in sound.features.features if abs(feat.value) > 0.1)
            analysis['feature_complexity'] += active_features
    
    # Calculate average complexity
    if analysis['total_sounds'] > 0:
        analysis['avg_feature_complexity'] = analysis['feature_complexity'] / analysis['total_sounds']
    
    return analysis

# Analyze all inventories
typological_analyses = {}
for lang_type, inventory in language_inventories.items():
    analysis = analyze_inventory_typology(inventory)
    typological_analyses[lang_type] = analysis

# Create comparative table
comparison_data = []
for lang_type, analysis in typological_analyses.items():
    row = {
        'Language_Type': lang_type,
        'Total_Sounds': analysis['total_sounds'],
        'Consonants': analysis['consonants'],
        'Vowels': analysis['vowels'],
        'C/V_Ratio': analysis['consonants'] / max(analysis['vowels'], 1),
        'Places': len(analysis['places_of_articulation']),
        'Manners': len(analysis['manners_of_articulation']),
        'Avg_Complexity': analysis.get('avg_feature_complexity', 0),
        'Rare_Features': sum(analysis['rare_features'].values())
    }
    comparison_data.append(row)

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.round(2)

print("Cross-Linguistic Typological Analysis:")
print("=" * 40)
print(comparison_df.to_string(index=False))

# Visualize typological patterns
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Plot 1: Inventory size distribution
axes[0,0].bar(comparison_df['Language_Type'], comparison_df['Total_Sounds'], color='skyblue')
axes[0,0].set_title('Total Inventory Size')
axes[0,0].set_ylabel('Number of Sounds')
axes[0,0].tick_params(axis='x', rotation=45)

# Plot 2: C/V ratio
axes[0,1].bar(comparison_df['Language_Type'], comparison_df['C/V_Ratio'], color='lightcoral')
axes[0,1].set_title('Consonant/Vowel Ratio')
axes[0,1].set_ylabel('C/V Ratio')
axes[0,1].tick_params(axis='x', rotation=45)

# Plot 3: Feature complexity
axes[1,0].bar(comparison_df['Language_Type'], comparison_df['Avg_Complexity'], color='lightgreen')
axes[1,0].set_title('Average Feature Complexity')
axes[1,0].set_ylabel('Features per Sound')
axes[1,0].tick_params(axis='x', rotation=45)

# Plot 4: Rare features
axes[1,1].bar(comparison_df['Language_Type'], comparison_df['Rare_Features'], color='gold')
axes[1,1].set_title('Rare Feature Count')
axes[1,1].set_ylabel('Number of Rare Features')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 6. Phonological Universals Testing

Test various proposed phonological universals using the comprehensive Tresoldi system.

In [ ]:
def test_phonological_universals():
    """Test various phonological universal claims."""
    tresoldi_system = get_feature_system('tresoldi_distinctive')
    
    universals_tests = {}
    
    # Universal 1: If a language has /g/, it has /k/
    def test_voicing_implication():
        # This would need a database of language inventories
        # For demo, test with our sample inventories
        violations = 0
        total_langs = 0
        
        for lang_type, inventory in language_inventories.items():
            consonants = inventory['consonants']
            has_g = 'g' in consonants
            has_k = 'k' in consonants
            
            if has_g and not has_k:
                violations += 1
            total_langs += 1
        
        return {'violations': violations, 'total': total_langs, 'rate': violations/total_langs}
    
    # Universal 2: Voiceless stops are more common than voiced stops
    def test_voicing_frequency():
        voiceless_count = 0
        voiced_count = 0
        
        stops = ['p', 'b', 't', 'd', 'k', 'g', 'q', 'ʔ']
        
        for stop in stops:
            if tresoldi_system.has_grapheme(stop):
                sound = Sound(stop, 'tresoldi_distinctive')
                if (sound.get_feature_value('consonantal') > 0 and 
                    sound.get_feature_value('continuant') < 0):
                    if sound.get_feature_value('voice') > 0:
                        voiced_count += 1
                    elif sound.get_feature_value('voice') < 0:
                        voiceless_count += 1
        
        return {'voiceless': voiceless_count, 'voiced': voiced_count, 'ratio': voiceless_count/max(voiced_count,1)}
    
    # Universal 3: All languages have at least one nasal
    def test_nasal_universal():
        violations = 0
        total_langs = 0
        
        for lang_type, inventory in language_inventories.items():
            has_nasal = False
            
            for consonant in inventory['consonants']:
                if tresoldi_system.has_grapheme(consonant):
                    sound = Sound(consonant, 'tresoldi_distinctive')
                    if sound.get_feature_value('nasal') > 0:
                        has_nasal = True
                        break
            
            if not has_nasal:
                violations += 1
            total_langs += 1
        
        return {'violations': violations, 'total': total_langs, 'rate': violations/total_langs}
    
    # Universal 4: If a language has clicks, it has other stops
    def test_click_implication():
        violations = 0
        total_click_langs = 0
        
        for lang_type, inventory in language_inventories.items():
            has_clicks = False
            has_other_stops = False
            
            for consonant in inventory['consonants']:
                if tresoldi_system.has_grapheme(consonant):
                    sound = Sound(consonant, 'tresoldi_distinctive')
                    
                    if sound.has_feature('click') and sound.get_feature_value('click') > 0:
                        has_clicks = True
                    elif (sound.get_feature_value('consonantal') > 0 and 
                          sound.get_feature_value('continuant') < 0):
                        has_other_stops = True
            
            if has_clicks:
                total_click_langs += 1
                if not has_other_stops:
                    violations += 1
        
        return {'violations': violations, 'total': total_click_langs, 'rate': violations/max(total_click_langs,1)}
    
    # Run all tests
    universals_tests['Voicing_Implication'] = test_voicing_implication()
    universals_tests['Voicing_Frequency'] = test_voicing_frequency()
    universals_tests['Nasal_Universal'] = test_nasal_universal()
    universals_tests['Click_Implication'] = test_click_implication()
    
    return universals_tests

universal_results = test_phonological_universals()

print("Phonological Universals Testing:")
print("=" * 40)

print("1. Voicing Implication (if /g/ then /k/):")
voicing_impl = universal_results['Voicing_Implication']
print(f"   Violations: {voicing_impl['violations']}/{voicing_impl['total']} ({voicing_impl['rate']:.1%})")

print("\n2. Voicing Frequency (voiceless > voiced stops):")
voicing_freq = universal_results['Voicing_Frequency']
print(f"   Voiceless stops: {voicing_freq['voiceless']}")
print(f"   Voiced stops: {voicing_freq['voiced']}")
print(f"   Ratio: {voicing_freq['ratio']:.2f}")

print("\n3. Nasal Universal (all languages have nasals):")
nasal_univ = universal_results['Nasal_Universal']
print(f"   Violations: {nasal_univ['violations']}/{nasal_univ['total']} ({nasal_univ['rate']:.1%})")

print("\n4. Click Implication (if clicks then other stops):")
click_impl = universal_results['Click_Implication']
print(f"   Violations: {click_impl['violations']}/{click_impl['total']} ({click_impl['rate']:.1%})")

## 7. Advanced Gradient Sound Change Modeling

Model realistic sound change with continuous feature values and variable application.

In [ ]:
def model_realistic_lenition():
    """Model realistic lenition process with gradient application."""
    engine = SoundChangeEngine(feature_system_name='unified_distinctive')
    
    # Progressive lenition stages
    lenition_stages = [
        # Stage 1: Initial weakening (20% progress)
        FeatureChangeRule(
            name="lenition_stage1",
            feature_conditions={"consonantal": 1.0, "continuant": -1.0, "sonorant": -1.0},
            feature_changes=[
                FeatureChange(
                    feature_name="continuant",
                    target_value=0.2,
                    change_type=ChangeType.GRADIENT,
                    change_strength=0.3
                )
            ],
            environment=PhonologicalEnvironment(left_pattern="V", right_pattern="V")
        ),
        
        # Stage 2: Further weakening (50% progress)
        FeatureChangeRule(
            name="lenition_stage2",
            feature_conditions={"consonantal": 1.0, "continuant": 0.2},
            feature_changes=[
                FeatureChange(
                    feature_name="continuant",
                    target_value=0.6,
                    change_type=ChangeType.GRADIENT,
                    change_strength=0.5
                )
            ]
        ),
        
        # Stage 3: Complete fricativization (100% progress)
        FeatureChangeRule(
            name="lenition_stage3",
            feature_conditions={"consonantal": 1.0, "continuant": 0.6},
            feature_changes=[
                FeatureChange(
                    feature_name="continuant",
                    target_value=1.0,
                    change_type=ChangeType.GRADIENT,
                    change_strength=0.8
                )
            ]
        )
    ]
    
    lenition_process = RuleSet(rules=lenition_stages, iterative=True, max_iterations=5)
    
    # Test words with intervocalic stops
    test_words = [
        ['a', 'p', 'a'],
        ['e', 't', 'a'],  
        ['i', 'k', 'o']
    ]
    
    print("Gradient Lenition Process:")
    print("=" * 30)
    
    for word in test_words:
        sounds = [Sound(s, 'unified_distinctive') for s in word]
        original = ' '.join([s.grapheme() for s in sounds])
        
        # Track lenition progress
        current_sounds = sounds
        print(f"\nWord: {original}")
        
        # Apply lenition process
        result = engine.apply_rule_set(lenition_process, sounds)
        
        # Show final state
        final = ' '.join([s.grapheme() for s in result.final_sequence])
        
        # Show continuancy values for middle consonant
        if len(result.final_sequence) >= 2:
            middle_consonant = result.final_sequence[1]
            continuant_val = middle_consonant.get_feature_value('continuant')
            print(f"  Final: {final} (continuant = {continuant_val:.2f})")
            
            # Interpret gradient value
            if continuant_val < 0.3:
                interpretation = "stop"
            elif continuant_val < 0.7:
                interpretation = "weakened stop / affricate"
            else:
                interpretation = "fricative"
            print(f"  Interpretation: {interpretation}")

model_realistic_lenition()

# Visualize lenition trajectory
def plot_lenition_trajectory():
    """Plot the trajectory of lenition over time."""
    # Simulate lenition progress over time
    time_steps = np.linspace(0, 100, 21)  # 0 to 100 years
    continuant_values = []
    
    for t in time_steps:
        # Sigmoid-like progression
        progress = 1 / (1 + np.exp(-(t - 50) / 15))
        continuant_val = -1.0 + 2.0 * progress  # Scale from -1 to +1
        continuant_values.append(continuant_val)
    
    plt.figure(figsize=(10, 6))
    plt.plot(time_steps, continuant_values, 'b-', linewidth=2, label='Continuant feature value')
    plt.axhline(y=-1, color='r', linestyle='--', alpha=0.5, label='Stop (-1.0)')
    plt.axhline(y=0, color='orange', linestyle='--', alpha=0.5, label='Intermediate (0.0)')
    plt.axhline(y=1, color='g', linestyle='--', alpha=0.5, label='Fricative (+1.0)')
    
    plt.xlabel('Time (arbitrary units)')
    plt.ylabel('Continuant Feature Value')
    plt.title('Gradient Lenition: Stop → Fricative Trajectory')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

plot_lenition_trajectory()

## 8. Summary and Conclusions

This notebook has demonstrated the full range of AlteruPhono's capabilities for historical linguistics research.

In [ ]:
# Summary of workflow capabilities
print("AlteruPhono Historical Linguistics Workflow Summary")
print("=" * 55)
print()

capabilities = [
    "✓ Comparative reconstruction from cognate sets",
    "✓ Language evolution simulation with sound change rules", 
    "✓ Phonological distance calculation for classification",
    "✓ Automatic sound change rule discovery",
    "✓ Cross-linguistic typological analysis",
    "✓ Phonological universals testing",
    "✓ Gradient sound change modeling",
    "✓ Feature-based phonological analysis",
    "✓ Multi-system compatibility (IPA, Unified, Tresoldi)",
    "✓ Large-scale cross-linguistic coverage (1,081 sounds)"
]

for capability in capabilities:
    print(capability)

print("\nKey Research Applications:")
print("-" * 30)
applications = [
    "• Comparative method automation",
    "• Language family classification", 
    "• Sound change pathway modeling",
    "• Phonological typology studies",
    "• Historical phonology reconstruction",
    "• Language contact analysis",
    "• Evolutionary linguistics simulation"
]

for application in applications:
    print(application)

print("\nNext Steps for Research:")
print("-" * 25)
next_steps = [
    "1. Apply to larger datasets with real language families",
    "2. Integrate with phylogenetic reconstruction methods",
    "3. Develop machine learning models for change prediction",
    "4. Create comprehensive typological databases",
    "5. Model contact-induced changes and borrowing",
    "6. Validate against known historical changes"
]

for step in next_steps:
    print(step)

print("\n" + "=" * 55)
print("AlteruPhono: Empowering Historical Linguistics Research")
print("=" * 55)